<a href="https://colab.research.google.com/github/onelgo/onelgo/blob/main/h1n1_flu_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

In [ ]:
data = pd.read_csv('/content/Flu_Shot_Learning_Predict_H1N1_and_Seasonal_Flu_Vaccines_-_Training_Features.csv')
data.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [ ]:
df_labels = pd.read_csv('/content/Flu_Shot_Learning_Predict_H1N1_and_Seasonal_Flu_Vaccines_-_Training_Labels.csv')
df_labels.head()

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0


In [ ]:
df_labels = df_labels.drop(['respondent_id'], axis=1)
df_labels.head()

,h1n1_vaccine,seasonal_vaccine
0,0,0
1,0,1
2,0,0
3,0,1
4,0,0


In [ ]:
df_labels.shape

(26707, 2)

In [ ]:
data.describe()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_adults,household_children
count,26707.000000,26615.000000,26591.000000,26636.000000,26499.000000,26688.000000,26665.000000,26620.00000,26625.000000,26579.000000,...,25903.000000,14433.00000,26316.000000,26319.000000,26312.000000,26245.000000,26193.000000,26170.000000,26458.000000,26458.000000
mean,13353.000000,1.618486,1.262532,0.048844,0.725612,0.068982,0.825614,0.35864,0.337315,0.677264,...,0.111918,0.87972,3.850623,2.342566,2.357670,4.025986,2.719162,2.118112,0.886499,0.534583
std,7709.791156,0.910311,0.618149,0.215545,0.446214,0.253429,0.379448,0.47961,0.472802,0.467531,...,0.315271,0.32530,1.007436,1.285539,1.362766,1.086565,1.385055,1.332950,0.753422,0.928173
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,6676.500000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,...,0.000000,1.00000,3.000000,1.000000,1.000000,4.000000,2.000000,1.000000,0.000000,0.000000
50%,13353.000000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,1.000000,...,0.000000,1.00000,4.000000,2.000000,2.000000,4.000000,2.000000,2.000000,1.000000,0.000000
75%,20029.500000,2.000000,2.000000,0.000000,1.000000,0.000000,1.000000,1.00000,1.000000,1.000000,...,0.000000,1.00000,5.000000,4.000000,4.000000,5.000000,4.000000,4.000000,1.000000,1.000000
max,26706.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,1.000000,1.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,3.000000


In [ ]:
df = data.drop(['respondent_id'], axis=1)
df.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [ ]:
df['doctor_recc_h1n1'].value_counts()

,count
doctor_recc_h1n1,
0.0,21299
1.0,5408


In [ ]:
df.dtypes

,0
h1n1_concern,float64
h1n1_knowledge,float64
behavioral_antiviral_meds,float64
behavioral_avoidance,float64
behavioral_face_mask,float64
behavioral_wash_hands,float64
behavioral_large_gatherings,float64
behavioral_outside_home,float64
behavioral_touch_face,float64
doctor_recc_h1n1,float64


In [ ]:
df.shape

(26707, 35)

In [ ]:
df['h1n1_concern'].isna().sum()

92

Checking missing values for numeric values (floats)

In [ ]:
# Iterate through each column and check for NA values
for col in df.columns:
  na_count = df[col].isna().sum()
  na_frequency = na_count / len(df) * 100
  if na_count > 0 and na_frequency < 4 and df[col].dtype=='float64': #impute columns with less than 4% missing values
    df[col].fillna(df[col].mode()[0], inplace=True)

<ipython-input-17-9b345fa849bd>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


In [ ]:
#checking whether above code worked
for col in df.columns:
  na_count = df[col].isna().sum()
  na_frequency = na_count / len(df) * 100
  if df[col].dtype=='float64':
    print (col, na_count, na_frequency)

h1n1_concern 0 0.0
h1n1_knowledge 0 0.0
behavioral_antiviral_meds 0 0.0
behavioral_avoidance 0 0.0
behavioral_face_mask 0 0.0
behavioral_wash_hands 0 0.0
behavioral_large_gatherings 0 0.0
behavioral_outside_home 0 0.0
behavioral_touch_face 0 0.0
doctor_recc_h1n1 2160 8.087767252031302
doctor_recc_seasonal 2160 8.087767252031302
chronic_med_condition 0 0.0
child_under_6_months 0 0.0
health_worker 0 0.0
health_insurance 12274 45.95798854232972
opinion_h1n1_vacc_effective 0 0.0
opinion_h1n1_risk 0 0.0
opinion_h1n1_sick_from_vacc 0 0.0
opinion_seas_vacc_effective 0 0.0
opinion_seas_risk 0 0.0
opinion_seas_sick_from_vacc 0 0.0
household_adults 0 0.0
household_children 0 0.0


In [ ]:
df[df['doctor_recc_seasonal'].isna()].head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
76,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"> $75,000",Not Married,Own,Employed,fpwskwrf,"MSA, Principle City",1.0,0.0,ldnlellj,xzmlyyjv
90,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,...,NaN,Not Married,Rent,Employed,lrircsnp,Non-MSA,0.0,0.0,fcxhlnwr,vlluhbov
94,2.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,kbazzjca,"MSA, Not Principle City",1.0,3.0,NaN,NaN
97,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,...,"> $75,000",Not Married,Own,Not in Labor Force,fpwskwrf,"MSA, Principle City",1.0,0.0,NaN,NaN


In [ ]:
df['doctor_recc_h1n1_missing'] = df['doctor_recc_h1n1'].isna().astype(int)
df['doctor_recc_seasonal_missing'] = df['doctor_recc_seasonal'].isna().astype(int)
df['health_insurance_missing'] = df['health_insurance'].isna().astype(int)
df.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,doctor_recc_h1n1_missing,doctor_recc_seasonal_missing,health_insurance_missing
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0,0
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,0,0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,1,1,1
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0,0,1
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0,1


In [ ]:
df['doctor_recc_h1n1'].fillna(df['doctor_recc_h1n1'].mode()[0], inplace=True)
df['doctor_recc_seasonal'].fillna(df['doctor_recc_seasonal'].mode()[0], inplace=True)
df['health_insurance'].fillna(df['health_insurance'].mode()[0], inplace=True)
df.head()

<ipython-input-23-a94cb2fda15a>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['doctor_recc_h1n1'].fillna(df['doctor_recc_h1n1'].mode()[0], inplace=True)
<ipython-input-23-a94cb2fda15a>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(va

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,doctor_recc_h1n1_missing,doctor_recc_seasonal_missing,health_insurance_missing
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0,0
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,0,0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,1,1,1
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0,0,1
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0,1


Now let's check the categorical values

In [ ]:
for col in df.columns:
  if df[col].dtype=='object':
    print (df[col].value_counts())

age_group
65+ Years        6843
55 - 64 Years    5563
45 - 54 Years    5238
18 - 34 Years    5215
35 - 44 Years    3848
Name: count, dtype: int64
education
College Graduate    10097
Some College         7043
12 Years             5797
< 12 Years           2363
Name: count, dtype: int64
race
White                21222
Black                 2118
Hispanic              1755
Other or Multiple     1612
Name: count, dtype: int64
sex
Female    15858
Male      10849
Name: count, dtype: int64
income_poverty
<= $75,000, Above Poverty    12777
> $75,000                     6810
Below Poverty                 2697
Name: count, dtype: int64
marital_status
Married        13555
Not Married    11744
Name: count, dtype: int64
rent_or_own
Own     18736
Rent     5929
Name: count, dtype: int64
employment_status
Employed              13560
Not in Labor Force    10231
Unemployed             1453
Name: count, dtype: int64
hhs_geo_region
lzgpxyit    4297
fpwskwrf    3265
qufhixun    3102
oxchjgsf    2859
kbazzjc

One hot encoding for categorical values

In [ ]:
cat_columns = df.select_dtypes(include='object').columns
df_encoded = pd.get_dummies(df, columns=cat_columns, drop_first=True)
df_encoded = df_encoded.astype(int)
df_encoded.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,employment_occupation_qxajmpny,employment_occupation_rcertsgn,employment_occupation_tfqavkke,employment_occupation_ukymxvdu,employment_occupation_uqqtjvyb,employment_occupation_vlluhbov,employment_occupation_xgwztkwe,employment_occupation_xqwwgdyp,employment_occupation_xtkaffoo,employment_occupation_xzmlyyjv
0,1,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,3,2,0,1,0,1,0,1,1,0,...,0,0,0,0,0,0,1,0,0,0
2,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,1,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,1,0,1,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


Scaling non-binary columns

In [ ]:
# Identify non-binary columns (assuming binary columns contain only 0s and 1s)
non_binary_cols = df.columns[(df.nunique() > 2)]
non_binary_cols = non_binary_cols[df[non_binary_cols].dtypes != 'object'] #filtering out categorical columns

scaler = MinMaxScaler()
df_encoded[non_binary_cols] = scaler.fit_transform(df_encoded[non_binary_cols])
df_encoded.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,employment_occupation_qxajmpny,employment_occupation_rcertsgn,employment_occupation_tfqavkke,employment_occupation_ukymxvdu,employment_occupation_uqqtjvyb,employment_occupation_vlluhbov,employment_occupation_xgwztkwe,employment_occupation_xqwwgdyp,employment_occupation_xtkaffoo,employment_occupation_xzmlyyjv
0,0.333333,0.0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1.000000,1.0,0,1,0,1,0,1,1,0,...,0,0,0,0,0,0,1,0,0,0
2,0.333333,0.5,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0.333333,0.5,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.666667,0.5,0,1,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


Creating the model

In [ ]:
X = df_encoded
y = df_labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=df_labels, random_state=42)

model = OneVsRestClassifier(LogisticRegression())
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

roc_auc_scores = {}
for i in range(y.shape[1]):
    roc_auc = roc_auc_score(y_test.iloc[:, i], y_pred_proba[:, i])
    roc_auc_scores[f'Label {i + 1}'] = roc_auc

# Display ROC AUC scores
for label, score in roc_auc_scores.items():
    print(f'{label} ROC AUC: {score:.4f}')
print("Average score:", roc_auc_score(y_test, y_pred_proba))

Label 1 ROC AUC: 0.8604
Label 2 ROC AUC: 0.8585
Average score: 0.8594477495681281


In [ ]:
print("Confusion Matrix for each label:")
for i in range(y.shape[1]):
    print(f"Label {i + 1}:")
    print(confusion_matrix(y_test.iloc[:, i], y_pred[:, i]))
    print("\nClassification Report:")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

Confusion Matrix for each label:
Label 1:
[[3995  217]
 [ 578  552]]

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.95      0.91      4212
           1       0.72      0.49      0.58      1130

    accuracy                           0.85      5342
   macro avg       0.80      0.72      0.75      5342
weighted avg       0.84      0.85      0.84      5342

Label 2:
[[2359  532]
 [ 618 1833]]

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.82      0.80      2891
           1       0.78      0.75      0.76      2451

    accuracy                           0.78      5342
   macro avg       0.78      0.78      0.78      5342
weighted avg       0.78      0.78      0.78      5342



In [ ]:
df_test = pd.read_csv('/content/Flu_Shot_Learning_Predict_H1N1_and_Seasonal_Flu_Vaccines_-_Test_Features.csv', index_col="respondent_id")
df_test.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,
26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird


In [ ]:
df_submission = pd.read_csv('/content/Flu_Shot_Learning_Predict_H1N1_and_Seasonal_Flu_Vaccines_-_Submission_Format.csv', index_col="respondent_id")
df_submission.head()

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.5,0.7
26708,0.5,0.7
26709,0.5,0.7
26710,0.5,0.7
26711,0.5,0.7
